# Carbon Monitoring Project

[FluxNet](http://fluxnet.fluxdata.org/) is a worldwide collection of sensor stations that record a number of local variables relating to atmospheric conditions, solar flux and soil moisture. This notebook visualizes the data used in the NASA Goddard/University of Alabama carbon monitoring project [NEE Data Fusion](https://github.com/greyNearing/nee_data_fusion/) (Grey Nearing et al., 2018), but using Python tools rather than Matlab.

The scientific goals of this notebook are to:

* examine the carbon flux measurements from each site (net C02 ecosystem exchange, or NEE)
* determine the feasibility of using a model to predict the carbon flux at one site from every other site.
* generate and explain model

The "meta" goal is to show how Python tools let you solve the scientific goals, so that you can apply these tools to your own problems.

In [ ]:
import sys
import dask
import numpy as np
import pandas as pd

import holoviews as hv

import hvplot.pandas
import geoviews.tile_sources as gts

pd.options.display.max_columns = 10
hv.extension('bokeh', width=80)

## Open the `intake` catalog
This notebook uses [`intake`](https://intake.readthedocs.io/) to set up a data catalog with instructions for loading data for various projects. Before we read in any data, we'll open that catalog file and inspect the various data sources:

In [ ]:
import intake

cat = intake.open_catalog('../catalog.yml')
list(cat)

## Load metadata
First we will load in the fluxnet_metadata containing some site information for each of the fluxnet sites. Included in these data are the lat and lon of each site and the vegetation encoding (more on this below). In the next cell we will read in these data and take a look at a random few lines:

In [ ]:
metadata = cat.fluxnet_metadata().read()
metadata.sample(5)

The vegetation type is classified according to the categories set out in the International Geosphere–Biosphere Programme (**igbd**) with several additional categories defined on the [fluxdata website](http://www.fluxdata.org/DataInfo/Dataset%20Doc%20Lib/VegTypeIGBP.aspx).

In [ ]:
igbp_vegetation = {
    'WAT': '00 - Water',
    'ENF': '01 - Evergreen Needleleaf Forest',
    'EBF': '02 - Evergreen Broadleaf Forest',
    'DNF': '03 - Deciduous Needleleaf Forest',
    'DBF': '04 - Deciduous Broadleaf Forest',
    'MF' : '05 - Mixed Forest',
    'CSH': '06 - Closed Shrublands',
    'OSH': '07 - Open shrublands',
    'WSA': '08 - Woody Savannas',
    'SAV': '09 - Savannas',
    'GRA': '10 - Grasslands',
    'WET': '11 - Permanent Wetlands',
    'CRO': '12 - Croplands',
    'URB': '13 - Urban and Built-up',
    'CNV': '14 - Cropland/Nartural Vegetation Mosaics',
    'SNO': '15 - Snow and Ice',
    'BSV': '16 - Baren or Sparsely Vegetated'
}

We can use the dictionary above to map from igbp codes to longer labels - creating a new column on our metadata. We will make this column an ordered categorical to improve visualizations.

In [ ]:
from pandas.api.types import CategoricalDtype

dtype = CategoricalDtype(ordered=True, categories=sorted(igbp_vegetation.values()))
metadata['vegetation'] = (metadata['igbp']
                          .apply(lambda x: igbp_vegetation[x])
                          .astype(dtype))
metadata.sample(5)

### Visualize the fluxdata sites

The PyViz ecosystem strives to make it always straightforward to visualize your data, to encourage you to be aware of it and understand it at each stage of a workflow. Here we will use Open Street Map tiles from `geoviews` 
to make a quick map of where the different sites are located and the vegetation at each site. 

In [ ]:
metadata.hvplot.points('lon', 'lat', geo=True, color='vegetation',
                       height=420, width=800, cmap='Category20') * gts.OSM

## Loading FluxNet data

The data in the [nee_data_fusion](https://github.com/greyNearing/nee_data_fusion/) repository is expressed as a collection of CSV files where the site names are expressed in the filenames.

This cell defines a function to:

* read in the data from all sites
* discard columns that we don't need
* calculate day of year
* caculate the season (spring, summer, fall, winter)

In [ ]:
data_columns = ['P_ERA', 'TA_ERA', 'PA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'WS_ERA',
                'VPD_ERA', 'TIMESTAMP', 'site', 'NEE_CUT_USTAR50']
soil_data_columns = ['SWC_F_MDS_1', 'SWC_F_MDS_2', 'SWC_F_MDS_3',
                     'TS_F_MDS_1', 'TS_F_MDS_2', 'TS_F_MDS_3']

keep_from_csv = data_columns + soil_data_columns

y_variable = 'NEE_CUT_USTAR50'

def season(df, metadata):
    """Add season column based on lat and month
    """
    site = df['site'].cat.categories.item()
    lat = metadata[metadata['site'] == site]['lat'].item()
    if lat > 0:
        seasons = {3: 'spring',  4: 'spring',  5: 'spring',
                   6: 'summer',  7: 'summer',  8: 'summer',
                   9: 'fall',   10: 'fall',   11: 'fall',
                  12: 'winter',  1: 'winter',  2: 'winter'}
    else:
        seasons = {3: 'fall',    4: 'fall',    5: 'fall',
                   6: 'winter',  7: 'winter',  8: 'winter',
                   9: 'spring', 10: 'spring', 11: 'spring',
                  12: 'summer',  1: 'summer',  2: 'summer'}
    return df.assign(season=df.TIMESTAMP.dt.month.map(seasons))

def clean_data(df):
    """
    Clean data columns:
    
    * add NaN col for missing columns
    * throw away un-needed columns
    * add day of year
    """
    df = df.assign(**{col: np.nan for col in keep_from_csv if col not in df.columns})
    df = df[keep_from_csv]
    
    df = df.assign(DOY=df.TIMESTAMP.dt.dayofyear)
    df = df.assign(year=df.TIMESTAMP.dt.year)
    df = season(df, metadata)
    
    return df

## Read and clean data

This will take a few minutes if the data is not cached yet. First we will get a list of all the files on the S3 bucket, then we will iterate over those files and cache, read, and munge the data in each one. This is necessary since the columns in each file don't necessarily match the columns in the other files. Before we concatenate across sites, we need to do some cleaning. 

In [ ]:
from s3fs import S3FileSystem

In [ ]:
s3 = S3FileSystem(anon=True)
s3_paths = s3.glob('earth-data/carbon_flux/nee_data_fusion/FLX*')

In [ ]:
datasets = []
skipped = []
used = []

for i, s3_path in enumerate(s3_paths):
    sys.stdout.write('\r{}/{}'.format(i+1, len(s3_paths)))
    
    dd = cat.fluxnet_daily(s3_path=s3_path).to_dask()
    site = dd['site'].cat.categories.item()
    
    if not set(dd.columns) >= set(data_columns):
        skipped.append(site)
        continue

    datasets.append(clean_data(dd))
    used.append(site)

print()
print('Found {} fluxnet sites with enough data to use - skipped {}'.format(len(used), len(skipped)))

Now that we have a list of datasets, we will concatenate across all rows. Since the data is loaded lazily - using `dask` - we need to explicitly call `compute` to get the data in memory. To learn more about this look at the [Data Ingestion](../tutorial/01_Data_Ingestion.ipynb) tutorial.

In [ ]:
data = dask.dataframe.concat(datasets).compute()
data.columns

We'll also set the data type of `'site'` to `'category'`. This will come in handy later.

In [ ]:
data['site'] = data['site'].astype('category')

### Visualizing Data Available at Sites

We can look at the sites for which we have data. We'll plot the sites on a world map again - this time using a custom colormap to denote sites with valid data, sites where data exist but were not loaded because too many fields were missing, and sites where no data was available. In addition to this map we'll get the count of different vegetation types at the sites.

In [ ]:
def mapper(x):
    if x in used:
        return 'valid'
    elif x in skipped:
        return 'skipped'
    else:
        return 'no data'
    
cmap = {'valid': 'green', 'skipped': 'red', 'no data': 'darkgray'}

QA = metadata.copy()
QA['quality'] = QA['site'].map(mapper)

all_points = QA.hvplot.points('lon', 'lat', geo=True, color='quality', 
                              cmap=cmap, hover_cols=['site', 'vegetation'],
                              height=420, width=600).options(tools=['hover', 'tap'], 
                                                             legend_position='top')

def veg_count(data):
    veg_count = data['vegetation'].value_counts().sort_index(ascending=False)
    return veg_count.hvplot.barh(height=420, width=500)

hist = veg_count(QA[QA.quality=='valid']).relabel('Vegetation counts for valid sites')

all_points * gts.OSM + hist

We'll make a couple of functions that generate plots on the full set of data or a subset of the data. We will use these in a dashboard below.

In [ ]:
def site_timeseries(data):
    """Timeseries plot showing the mean carbon flux at each DOY as well as the min and max"""
    
    tseries = hv.Overlay([
        (data.groupby(['DOY', 'year'])[y_variable]
             .mean().groupby('DOY').agg([np.min, np.max])
             .hvplot.area('DOY', 'amin', 'amax', alpha=0.2, fields={'amin': y_variable})),
        data.groupby('DOY')[y_variable].mean().hvplot()])
    
    return tseries.options(width=800, height=400)

def site_count_plot(data):
    """Plot of the number of observations of each of the non-mandatory variables."""
    return data[soil_data_columns + ['site']].count().hvplot.bar(rot=90, width=300, height=400)

timeseries = site_timeseries(data)
count_plot = site_count_plot(data)
timeseries + count_plot

### Dashboard

Using the plots and functions defined above, we can make a [Panel](http://panel.pyviz.org) dashboard of sites where by clicking on a site, you get the timeseries and variable count for that particular site.

In [ ]:
from holoviews.streams import Selection1D
import panel as pn

In [ ]:
stream = Selection1D(source=all_points)
empty = timeseries.relabel('No selection') + count_plot.relabel('No selection')

def site_selection(index):
    if not index:
        return empty
    i = index[0]
    if i in QA[QA.quality=='valid'].index:
        site = QA.iloc[i].site
        ts = site_timeseries(data[data.site == site]).relabel(site)
        ct = site_count_plot(data[data.site == site]).relabel(site)
        return ts + ct
    else:
        return empty

one_site = hv.DynamicMap(site_selection, streams=[stream])

pn.Column(pn.Row(all_points * gts.OSM, hist), pn.Row(one_site))

## Merge data

Now that the data are loaded in we can merge the daily data with the metadata from before.

In order to use the categorical `igbp` field with machine-learning tools, we will create a one-hot encoding where each column corresponds to one of the `igbp` types, the rows correspond to observations and all the cells are filled with 0 or 1. This can be done use the method `pd.get_dummies`:

In [ ]:
onehot_metadata = pd.get_dummies(metadata, columns=['igbp'])
onehot_metadata.sample(5)

We'll do the same for season - keeping season as a column. 

In [ ]:
data = pd.get_dummies(data, columns=['season']).assign(season=data['season'])

We'll merge the metadata with all our daily observations - creating a tidy dataframe. 

In [ ]:
df = pd.merge(data, onehot_metadata, on='site')
df.sample(5)

### Visualizing Soil Data Availability at Sites
Now that all of our observations are merged with the site metadata, we can take a look at which sites have soil data. Some sites have soil moisture and temperature data at one depths and others have the data at all 3 depths. We'll look at the distribution of availability across sites.

In [ ]:
partial_soil_data = df[df[soil_data_columns].notnull().any(1)]
partial_soil_data_sites = metadata[metadata.site.isin(partial_soil_data.site.unique())]

In [ ]:
full_soil_data = df[df[soil_data_columns].notnull().all(1)]
full_soil_data_sites = metadata[metadata.site.isin(full_soil_data.site.unique())]

In [ ]:
args = dict(geo=True, hover_cols=['site', 'vegetation'], height=420, width=600)

partial = partial_soil_data_sites.hvplot.points('lon', 'lat', **args).relabel('partial soil data')
full    =    full_soil_data_sites.hvplot.points('lon', 'lat', **args).relabel('full soil data')

(partial * full * gts.OSM).options(legend_position='top') +  veg_count(partial_soil_data_sites) * veg_count(full_soil_data_sites)

Since there seems to be a strong geographic pattern in the availablity of soil moisture and soil temperature data, we won't use those columns in our model. 

In [ ]:
df = df.drop(columns=soil_data_columns)

Now we will set data to only the rows where there are no null values:

In [ ]:
df = df[df.notnull().all(1)].reset_index(drop=True)

In [ ]:
df['site'] = df['site'].astype('category')

### Assigning roles to variables

Before we train a model to predict carbon flux globally we need to choose which variables will be included in the input to the model. For those we should only use variables that we expect to have some relationship with the variable that we are trying to predict. 

In [ ]:
explanatory_cols = ['lat']
data_cols = ['P_ERA', 'TA_ERA', 'PA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'WS_ERA', 'VPD_ERA']
season_cols = [col for col in df.columns if col.startswith('season_')]
igbp_cols = [col for col in df.columns if col.startswith('igbp_')]

In [ ]:
x = df[data_cols + igbp_cols + explanatory_cols + season_cols].values
y = df[y_variable].values

### Scaling the Data

In [ ]:
from sklearn.preprocessing import StandardScaler

# transform data matrix so 0 mean, unit variance for each feature
X = StandardScaler().fit_transform(x)

Now we are ready to train a model to predict carbon flux globally. 

## Training and Testing

We'll shuffle the sites and select 10% of them to be used as a test set. The rest we will use for training. Note that you might get better results using leave-one-out, but since we have a large amount of data, classical validation will be much faster.

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

sep = GroupShuffleSplit(train_size=0.9, test_size=0.1)
train_idx, test_idx = next(sep.split(X, y, df.site.cat.codes.values))

In [ ]:
train_sites = df.site.iloc[train_idx].unique()
test_sites = df.site.iloc[test_idx].unique()

train_site_metadata = metadata[metadata.site.isin(train_sites)]
test_site_metadata = metadata[metadata.site.isin(test_sites)]

Let's make a world map showing the sites that will be used as in training and those that will be used in testing:

In [ ]:
train = train_site_metadata.hvplot.points('lon', 'lat', **args).relabel('training sites')
test  = test_site_metadata.hvplot.points( 'lon', 'lat', **args).relabel('testing sites') 

(train * test * gts.OSM).options(legend_position='top') +  veg_count(train_site_metadata) * veg_count(test_site_metadata)

This distribution seems reasonably uniform and unbiased, though a different random sampling might have allowed testing for each continent and all vegetation types.

## Training the Regression Model

We'll construct a linear regression model using our randomly selected training sites and test sites.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
model.fit(X[train_idx], y[train_idx]);

We'll create a little function to look at observed vs predicted values

In [ ]:
from holoviews.operation.datashader import datashade

def result_plot(predicted, observed, title, corr=None, res=0.1):
    """Plot datashaded observed vs predicted"""
    
    corr = corr if corr is not None else np.corrcoef(predicted, observed)[0][1]
    title = '{} (correlation: {:.02f})'.format(title, corr)
    scatter = hv.Scatter((predicted, observed), 'predicted', 'observed')\
                .redim.range(predicted=(observed.min(), observed.max()))
    
    return datashade(scatter, y_sampling=res, x_sampling=res).relabel(title)

In [ ]:
(result_plot(model.predict(X[train_idx]), y[train_idx], 'Training') + \
 result_plot(model.predict(X[test_idx ]), y[test_idx],  'Testing')).options('RGB', axiswise=True, width=500)

### Prediction at test sites

We can see how well the prediction does at each of our testing sites by making another dashboard. 

In [ ]:
results = []

for site in test_sites:
    site_test_idx = df[df.site == site].index
    y_hat_test = model.predict(X[site_test_idx])
    corr =  np.corrcoef(y_hat_test, y[site_test_idx])[0][1]
    
    results.append({'site': site,
                    'observed': y[site_test_idx], 
                    'predicted': y_hat_test, 
                    'corr': corr})

In [ ]:
test_site_results = pd.merge(test_site_metadata, pd.DataFrame(results), 
                             on='site').set_index('site', drop=False)

Now we can set up another dashboard with just the test sites, where tapping on a given site produces a plot of the predicted vs. observed carbon flux.

First we'll set up a timeseries function.

In [ ]:
def timeseries_observed_vs_predicted(site=None):
    """
    Make a timeseries plot showing the predicted/observed 
    mean carbon flux at each DOY as well as the min and max
    """
    if site:
        data = df[df.site == site].assign(predicted=test_site_results.loc[site, 'predicted'])
        corr = test_site_results.loc[site, 'corr']
        title = 'Site: {}, correlation coefficient: {:.02f}'.format(site, corr)
    else:
        data = df.assign(predicted=np.nan)
        title = 'No Selection'

    spread = data.groupby(['DOY', 'year'])[y_variable].mean().groupby('DOY').agg([np.min, np.max]) \
             .hvplot.area('DOY', 'amin', 'amax', alpha=0.2, fields={'amin': 'observed'})
    observed  = data.groupby('DOY')[y_variable ].mean().hvplot().relabel('observed')
    predicted = data.groupby('DOY')['predicted'].mean().hvplot().relabel('predicted')
    
    return (spread * observed * predicted).options(width=800).relabel(title)

In [ ]:
timeseries_observed_vs_predicted(test_sites[1])

Then we'll set up the points colored by correlation coefficient.

In [ ]:
test_points = test_site_results.hvplot.points('lon', 'lat', geo=True, c='corr', legend=False,
                                              cmap='coolwarm_r', s=150, height=420, width=800, 
                                              hover_cols=['vegetation', 'site']).options(
                                              tools=['tap', 'hover'], line_color='black')

And put it together into a dashboard. This will look very similar to the one above.

In [ ]:
test_stream = Selection1D(source=test_points)

def test_site_selection(index):
    site = None if not index else test_sites[index[0]]
    return timeseries_observed_vs_predicted(site)

one_test_site = hv.DynamicMap(test_site_selection, streams=[test_stream])
title = 'Test sites colored by correlation: tap on site to plot long-term-mean timeseries'

dash = pn.Column((test_points * gts.OSM).relabel(title), one_test_site)
dash.servable()

## Optional: Seasonal Prediction

Clicking on some of the sites above suggests that prediction often works well for some months and not for others. Perhaps different variables are important for prediction, depending on the season? We might be able to achieve better results if we generate separate models for each season. First we'll set up a function that computes prediction stats for a given training index, test index, array of X, array of y and array of seasons.

In [ ]:
seasons = ['summer', 'fall', 'spring', 'winter']

In [ ]:
def prediction_stats(train_idx, test_idx, X, y, season):
    """
    Compute prediction stats for equal length arrays X, y, and season
    split into train_idx and test_idx
    """
    pred = {}

    for s in seasons:
        season_idx = np.where(season==s)
        season_train_idx = np.intersect1d(season_idx, train_idx, assume_unique=True)
        season_test_idx = np.intersect1d(season_idx, test_idx, assume_unique=True)
        
        model = LinearRegression()
        model.fit(X[season_train_idx], y[season_train_idx])
        
        y_hat = model.predict(X[season_test_idx])
        y_test = y[season_test_idx]
        pred[s] = {'predicted': y_hat,
                   'observed': y_test,
                   'corrcoef': np.corrcoef(y_hat, y_test)[0][1],
                   'test_index': test_idx}
    return pred

### Setup Dask
With dask, we can distribute tasks over cores and do parallel computation. For more information see https://dask.org/

In [ ]:
from distributed import Client

client = Client()
client

Now we'll scatter our data using `dask` and make a bunch of different splits. For each split we'll compute the predicton stats for each season.

In [ ]:
futures = []
sep = GroupShuffleSplit(n_splits=50, train_size=0.9, test_size=0.1)

X_future = client.scatter(X)
y_future = client.scatter(y)
season_future = client.scatter(df['season'].values)

for i, (train_index, test_index) in enumerate(sep.split(X, y, df.site.cat.codes.values)):
    train_future = client.scatter(train_index)
    test_future = client.scatter(test_index)
    futures += [client.submit(prediction_stats, train_future, test_future,
                              X_future, y_future, season_future)]

Now that we have our computations set up in dask, we can gather the results:

In [ ]:
results = client.gather(futures)

And consolidate the results for each season.

In [ ]:
output = {
    s: {
        'predicted': np.concatenate([i[s]['predicted'] for i in results]),
        'observed': np.concatenate([i[s]['observed'] for i in results]),
        'test_index': np.concatenate([i[s]['test_index'] for i in results]),
        'corrcoef': np.array([i[s]['corrcoef'] for i in results])
    } for s in seasons}

In [ ]:
hv.Layout([
    result_plot(output[s]['predicted'], output[s]['observed'], s, output[s]['corrcoef'].mean())
    for s in seasons]).cols(2).options('RGB', axiswise=True, width=400)

In [ ]:
def helper(s):
    corr = output[s]['corrcoef']
    return pd.DataFrame([corr, [s] * len(corr)], index=['corr', 'season']).T

corr = pd.concat(map(helper, seasons)).reset_index(drop=True)

In [ ]:
corr.hvplot.hist(y='corr', groupby='season', bins=np.arange(0, .9, .05).tolist(), dynamic=False, width=500)

In [ ]:
corr.mean()

## Suggested Next Steps

 - Can we predict certain vegetations better than others?
 - Calculate fraction of explained variance.
 - replace each FluxNet input variable with a remotely sensed (satellite imaged) quantity to predict carbon flux globally